In [ ]:
%pip install geopandas matplotlib plotly nbformat notebook ipywidgets

In [1]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load the Natural Earth shapefile (adjust path as needed)
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

# Filter for Europe
europe = world[world['CONTINENT'] == 'Europe']

# Sample CO2 data (adjust with real data if available)
emission_data = {
    'ADMIN': [
        'Germany', 'France', 'United Kingdom', 'Italy', 'Spain',
        'Poland', 'Netherlands', 'Belgium', 'Sweden', 'Austria'
    ],
    'co2_2019': [750, 340, 380, 320, 270, 320, 170, 120, 45, 65],
    'co2_2021': [670, 310, 340, 300, 250, 290, 160, 110, 44, 63]
}
emissions_df = pd.DataFrame(emission_data)

# Merge emissions with GeoDataFrame
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN')
europe_emissions.fillna({'co2_2019': 0, 'co2_2021': 0}, inplace=True)

# Add ID for Plotly
europe_emissions['id'] = europe_emissions['ADMIN']
geojson = europe_emissions.__geo_interface__

# Create Plotly choropleth
fig = px.choropleth(
    europe_emissions,
    geojson=geojson,
    locations='id',
    color='co2_2019',
    hover_name='ADMIN',
    color_continuous_scale="Reds",
    featureidkey="properties.ADMIN",
    title="CO₂ Emissions in Europe (2019 vs 2021)",
)

# Add dropdown to switch years
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{"color": [europe_emissions["co2_2019"]]}],
                    label="2019 (Pre-Pandemic)",
                    method="restyle"
                ),
                dict(
                    args=[{"color": [europe_emissions["co2_2021"]]}],
                    label="2021 (Post-Pandemic)",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


Test Text